In [1]:
import os
os.chdir('../')

In [2]:
import data
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import preprocess_utils.session2vec as sess2vec
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
train_df = data.train_df('small')
test_df = data.test_df('small')
targets = data.target_indices('small')

full_df = pd.concat([train_df, test_df]).drop(['platform','city','current_filters'],axis=1)
print(full_df.shape)

(5011, 9)


In [4]:
full_df.tail()

,user_id,session_id,timestamp,step,action_type,reference,device,impressions,prices
4988,ARM28VZ4VBX8,65da983c12545,1541109189,11,clickout item,NaN,desktop,2282646|42352|513161|3239488|2809306|2042953|4...,56|65|54|48|23|47|46|26|28|21|35|17|36|30|48|2...
4989,ARM28VZ4VBX8,65da983c12545,1541109664,12,interaction item image,2282646,desktop,NaN,NaN
4990,ARM28VZ4VBX8,65da983c12545,1541109664,13,interaction item image,2282646,desktop,NaN,NaN
4991,ARM28VZ4VBX8,65da983c12545,1541109664,14,interaction item info,2282646,desktop,NaN,NaN
4992,ARM28VZ4VBX8,65da983c12545,1541109869,15,interaction item rating,157658,desktop,NaN,NaN


In [5]:
def add_impressions_columns_as_new_actions(df):
    df['impression_price'] = np.nan
    clickout_rows = df.loc[(df[df.action_type == 'clickout item'].index)]
    
    temp_df = []
    
    for _, row in tqdm(clickout_rows.iterrows()):
        impressions = list(map(int, row.impressions.split('|')))
        prices = list(map(int, row.prices.split('|')))
        row.action_type = 'show_impression'
  
        steps = np.linspace(row.step-1+1/len(impressions),row.step,len(impressions)+1)
        for imprsn, impr_price, step in zip(impressions, prices, steps):
            row.reference = imprsn
            row.impression_price = impr_price
            row.step = step
            #row.original_index = -1   # the show_impression-type rows indices is -1
            temp_df.append(tuple(row))

    temp_df = pd.DataFrame(temp_df, columns=clickout_rows.columns)
    df = df.append(temp_df, ignore_index=True).drop(['impressions','prices'],axis=1)
    df.index = df.index.set_names(['index'])
    #print(df.columns)
    return df.sort_values(['user_id','session_id','timestamp','step'])

In [6]:
full_df = add_impressions_columns_as_new_actions(full_df)

In [7]:
full_df

,user_id,session_id,timestamp,step,action_type,reference,device,impression_price
index,,,,,,,,
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1.0000,search for poi,Newtown,mobile,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2.0000,interaction item image,666856,mobile,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3.0000,interaction item image,666856,mobile,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4.0000,interaction item image,666856,mobile,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5.0000,interaction item image,109038,mobile,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6.0000,interaction item image,666856,mobile,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7.0000,interaction item image,109038,mobile,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8.0000,interaction item image,666856,mobile,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9.0000,interaction item image,109038,mobile,NaN


In [8]:
full_df.index.values

array([    0,     1,     2, ..., 14325, 14326,  3676])

In [12]:
#full_df[(full_df.user_id == '00RL8Z82B2Z1') & (full_df.session_id == 'aff3928535f48')].sort_values(['timestamp','step'])

In [5]:
accomodations_df = data.accomodations_df()

In [44]:
def get_accomodations_one_hot(accomodations_df):
    # one-hot encoding of the accomodations features
    tqdm.pandas()
    accomodations_df.properties = accomodations_df.properties.progress_apply(
        lambda x: x.split('|') if isinstance(x, str) else x)
    accomodations_df.fillna(value='', inplace=True)

    mlb = MultiLabelBinarizer()
    one_hot_features = mlb.fit_transform(accomodations_df.properties)
    one_hot_accomodations_df = pd.DataFrame(one_hot_features, columns=mlb.classes_)

    # create dataframe with columns: item_id  |  one_hot_features[...]
    attributes_df = pd.concat([accomodations_df.drop('properties', axis=1), one_hot_accomodations_df], axis=1)
    return attributes_df

In [45]:
attributes_df = get_accomodations_one_hot(accomodations_df)

In [47]:
attributes_df.head()

,item_id,,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,...,Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms)
0,5101,0,0,0,0,1,0,1,1,0,...,1,0,0,0,0,0,0,0,1,1
1,5416,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,1,1,1
2,5834,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,1
3,5910,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,6066,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,0,0,1,1,1


In [85]:
def df2vec(df, accomodations_df):
    # one-hot encoding of the accomodations features
    #attributes_df = get_accomodations_one_hot(df, accomodations_df)
    features_columns_count = attributes_df.shape[1] -1
    
    # substitute a dataframe column with its one-hot encoding derived columns
    def one_hot_df_column(df, column_label):
        return pd.concat([df, pd.get_dummies(df[column_label], prefix=column_label)], axis=1).drop([column_label],axis=1)

    # add the one-hot encoding of the action_type feature
    df = one_hot_df_column(df, 'action_type')

    # add the one-hot encoding of the device feature
    df = one_hot_df_column(df, 'device')

    # add the 'no-reference' column
    #df['no_reference'] = (~df.reference.fillna('').str.isnumeric()) * 1
    
    # add the one-hot encoding columns of the reference accomodation
    df = df.astype({'reference':str}).reset_index().merge(attributes_df.astype({'item_id':str}), how='left', left_on='reference',right_on='item_id').set_index('index')
    # set all the one-hot features to 0 for the non-reference rows
    accomodations_features_columns = attributes_df.drop('item_id', axis=1).columns
    df[accomodations_features_columns] = df[accomodations_features_columns].fillna(value=0).astype(int)

    return df, df.columns[-features_columns_count:], attributes_df

In [86]:
res, _, _ = df2vec(full_df, accomodations_df)

In [87]:
res

,user_id,session_id,timestamp,step,reference,impression_price,action_type_change of sort order,action_type_clickout item,action_type_filter selection,action_type_interaction item deals,action_type_interaction item image,action_type_interaction item info,action_type_interaction item rating,action_type_search for destination,action_type_search for item,action_type_search for poi,action_type_show_impression,device_desktop,device_mobile,device_tablet,item_id,,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,Air Conditioning,Airport Hotel,Airport Shuttle,All Inclusive (Upon Inquiry),Balcony,Bathtub,Beach,Beach Bar,Beauty Salon,Bed & Breakfast,Bike Rental,Boat Rental,Body Treatments,Boutique Hotel,Bowling,Bungalows,Business Centre,Business Hotel,Cable TV,Camping Site,Car Park,Casa Rural (ES),Casino (Hotel),Central Heating,Childcare,Club Hotel,Computer with Internet,Concierge,Conference Rooms,Convenience Store,Convention Hotel,Cosmetic Mirror,Cot,Country Hotel,Deck Chairs,Design Hotel,Desk,Direct beach access,Diving,Doctor On-Site,Eco-Friendly hotel,Electric Kettle,Excellent Rating,Express Check-In / Check-Out,Family Friendly,Fan,Farmstay,Fitness,Flatscreen TV,Free WiFi (Combined),Free WiFi (Public Areas),Free WiFi (Rooms),Fridge,Gay-friendly,Golf Course,Good Rating,Guest House,Gym,Hairdresser,Hairdryer,Halal Food,Hammam,Health Retreat,Hiking Trail,Honeymoon,Horse Riding,Hostal (ES),Hostel,Hot Stone Massage,Hotel,Hotel Bar,House / Apartment,Hydrotherapy,Hypoallergenic Bedding,Hypoallergenic Rooms,Ironing Board,Jacuzzi (Hotel),Kids' Club,Kosher Food,Large Groups,Laundry Service,Lift,Luxury Hotel,Massage,Microwave,Minigolf,Motel,Nightclub,Non-Smoking Rooms,On-Site Boutique Shopping,Openable Windows,Organised Activities,Pet Friendly,Playground,Pool Table,Porter,Pousada (BR),Radio,Reception (24/7),Resort,Restaurant,Romantic,Room Service,Room Service (24/7),Safe (Hotel),Safe (Rooms),Sailing,Satellite TV,Satisfactory Rating,Sauna,Self Catering,Senior Travellers,Serviced Apartment,Shooting Sports,Shower,Singles,Sitting Area (Rooms),Ski Resort,Skiing,Solarium,Spa (Wellness Facility),Spa Hotel,Steam Room,Sun Umbrellas,Surfing,Swimming Pool (Bar),Swimming Pool (Combined Filter),Swimming Pool (Indoor),Swimming Pool (Outdoor),Szep Kartya,Table Tennis,Telephone,Teleprinter,Television,Tennis Court,Tennis Court (Indoor),Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms)
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1.0000,Newtown,NaN,0,0,0,0,0,0,0,0,0,1,0,0,1,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2.0000,666856,NaN,0,0,0,0,1,0,0,0,0,0,0,0,1,0,666856,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,1
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3.0000,666856,NaN,0,0,0,0,1,0,0,0,0,0,0,0,1,0,666856,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,1
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4.0000,666856,NaN,0,0,0,0,1,0,0,0,0,0,0,0,1,0,66685